# Kasus 3: Kapasitas Kontainer dan Informasi Pelabuhan
---

## Entity Relationship Diagram (ERD)

In [1]:
import base64
from IPython.display import Image, display

# credits: https://gist.github.com/MLKrisJohnson/2d2df47879ee6afd3be9d6788241fe99
def mm_ink(graphbytes):
  """Given a bytes object holding a Mermaid-format graph, return a URL that will generate the image."""
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("ascii")
  return "https://mermaid.ink/img/" + base64_string

def mm_display(graphbytes):
  """Given a bytes object holding a Mermaid-format graph, display it."""
  display(Image(url=mm_ink(graphbytes)))

def mm(graph):
  """Given a string containing a Mermaid-format graph, display it."""
  graphbytes = graph.encode("ascii")
  mm_display(graphbytes)

def mm_link(graph):
  """Given a string containing a Mermaid-format graph, return URL for display."""
  graphbytes = graph.encode("ascii")
  return mm_ink(graphbytes)
  
def mm_path(path):
  """Given a path to a file containing a Mermaid-format graph, display it"""
  with open(path, 'rb') as f:
    graphbytes = f.read()
  mm_display(graphbytes)

mm("""
erDiagram
    port_details ||--|{ port_container_capacity : consists_of
    port_details {
        INT port_id PK
        VARCHAR port_name
        VARCHAR location
    }
    port_container_capacity {
        INT id PK
        INT port_id FK
        INT year
        INT month
        INT capacity
    }
""")

## Load Sample Data into an SQLite Database

In [2]:
import pandas as pd
from sqlite3 import connect
from IPython.display import HTML

# Helper class to display tables side-by-side
# credits: https://python.plainenglish.io/displaying-multiple-dataframes-side-by-side-in-jupyter-lab-notebook-9a4649a4940
def side_by_side(*dfs):
  html = '<div style="display:flex">'
  for df in dfs:
      html += '<div style="margin-right: 2em">'
      html += df.to_html()
      html += '</div>'
  html += '</div>'
  display(HTML(html))

# Load data from CSV files
port_details_file_path = './3-port_details.csv'
port_capacity_file_path = './3-port_container_capacity.csv'
details_data = pd.read_csv(port_details_file_path)
capacity_data = pd.read_csv(port_capacity_file_path)

# Create an in-memory SQLite database
conn = connect(':memory:')
details_data.to_sql(name='details', con=conn)
capacity_data.to_sql(name='capacities', con=conn)

side_by_side(details_data,capacity_data)

# Task 1
> 1. Tulis query untuk menghasilkan nama lokasi, kuartal (Q1, Q2, Q3, Q4), dan total kapasitas
kontainer.

In [3]:
pd.read_sql('SELECT * from details', conn)

,index,port_id,port_name,location
0,0,1,Belawan,North Sumatra
1,1,2,Tanjung Priok,Jakarta
